# ABCD Yields

In [3]:
import ROOT as r
import pandas as pd
import numpy as np

In [14]:
def get_abcd(file, BDT_CUT=0.56, DNN_CUT=0.92):
    rdf = r.RDataFrame("Events", file)
    df = pd.DataFrame(rdf.AsNumpy(["weight", "VBSBDTscore", "abcdnet_score", "Hbbscore", "HighestWjetScore"]))

    df = df.query("Hbbscore > 0.5 & HighestWjetScore > 0.7")

    a = df.query("VBSBDTscore > @BDT_CUT & abcdnet_score > @DNN_CUT").weight
    b = df.query("VBSBDTscore > @BDT_CUT & abcdnet_score < @DNN_CUT").weight
    c = df.query("VBSBDTscore < @BDT_CUT & abcdnet_score > @DNN_CUT").weight
    d = df.query("VBSBDTscore < @BDT_CUT & abcdnet_score < @DNN_CUT").weight

    err_a = np.sqrt((a**2).sum())
    err_b = np.sqrt((b**2).sum())
    err_c = np.sqrt((c**2).sum())
    err_d = np.sqrt((d**2).sum())

    a = a.sum()
    b = b.sum()
    c = c.sum()
    d = d.sum()

    if "data_MVA" in file:
        a = np.array(1)

    print(f"Actual A: {a} \\pm {err_a}")
    print(f"Region B: {b} \\pm {err_b}")
    print(f"Region C: {c} \\pm {err_c}")
    print(f"Region D: {d} \\pm {err_d}")

    err_pwhite_a = np.sqrt((err_b * c / d) ** 2 + (err_c * b / d) ** 2 + ((b * c / d**2) * err_d) ** 2)

    print(f"Predicted A: {b * c / d} \\pm {err_pwhite_a}")

# Yields

In [17]:
cuts = {
    "BDT_CUT": 0.9,
    "DNN_CUT": 0.88
}

In [22]:
print("###### DATA #######")
data = get_abcd("/data/userdata/aaarora/output/run2/data_MVA.root", **cuts)

print("###### SIG #######")
sig = get_abcd("/data/userdata/aaarora/output/run2/sig_MVA.root", **cuts)

print("###### BKG #######")
bkg = get_abcd("/data/userdata/aaarora/output/run2/bkg_MVA.root", **cuts)

###### DATA #######
Actual A: 1 \pm 1.0
Region B: 75.0 \pm 8.660254037844387
Region C: 29.0 \pm 5.385164807134504
Region D: 998.0 \pm 31.591137997862628
Predicted A: 2.17935871743487 \pm 0.4815253309236473
###### SIG #######
Actual A: 4.008916583563946 \pm 0.026121250747068222
Region B: 0.917439074610472 \pm 0.013804310108913389
Region C: 0.8548453812651182 \pm 0.012684376878089548
Region D: 0.42313491642532114 \pm 0.009442712404493465
Predicted A: 1.853471611722502 \pm 0.05696460880366702
###### BKG #######
Actual A: 1.8839609718510455 \pm 0.39070884589657756
Region B: 71.35323669488197 \pm 3.6031327900855388
Region C: 25.288997187074596 \pm 2.146817255646129
Region D: 945.3695745387407 \pm 10.558591916674988
Predicted A: 1.9087263337684162 \pm 0.18973586603674344


In [19]:
sign = lambda s, b_a:  np.sqrt(2 * ((s + b_a) * np.log(1 + s / b_a) - s))

In [23]:
sign(3.651707521299207, 1.6393447198915654)

2.2573998535935083